<a href="https://colab.research.google.com/github/boyuedong/Personal-Projects/blob/main/sentiment_analysis_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install requests beautifulsoup4 pymongo


In [12]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import urljoin

# Target URL
URL = "https://finance.yahoo.com/news/"
headers = {"User-Agent": "Mozilla/5.0"}

# Send request
response = requests.get(URL, headers=headers)
soup = BeautifulSoup(response.text, "html.parser")

# Extract news headlines and links
articles = soup.select("h3 a")  # Yahoo Finance article headlines

news_data = []

for article in articles:
    title = article.text
    url = urljoin("https://finance.yahoo.com", article["href"])  # ✅ Fix URL bug

    # Visit the article page
    article_response = requests.get(url, headers=headers)
    article_soup = BeautifulSoup(article_response.text, "html.parser")

    paragraphs = article_soup.find_all("p")
    content = " ".join([p.text for p in paragraphs])  # Extract article text

    news_data.append({"title": title, "url": url, "content": content})

# Save to CSV
df = pd.DataFrame(news_data)
df.to_csv("finance_news.csv", index=False)

# Display first few results
df.head()


,title,url,content
0,News,https://www.yahoo.com/,Manage your account 5 min ago Subscribe to...
1,Life,https://www.yahoo.com/lifestyle/,"Making friends as an adult can be challenging,..."
2,Entertainment,https://www.yahoo.com/entertainment/,Daniel Craig isn’t enlisting in the DC Univers...
3,Finance,https://finance.yahoo.com/,President Donald Trump suggested that savings ...
4,Sports,https://sports.yahoo.com/,On this episode of The Big Number Tom Haberstr...


In [14]:
df.to_csv("finance_news.csv", index=False)
print("✅ Data saved as finance_news.csv")


✅ Data saved as finance_news.csv


In [16]:
from huggingface_hub import login
login("your_huggingface_token_here")


HTTPError: Invalid user token. If you didn't pass a user token, make sure you are properly logged in by executing `huggingface-cli login`, and if you did pass a user token, double-check it's correct.

In [15]:
import pandas as pd
from transformers import pipeline

# Load the news data
df = pd.read_csv("finance_news.csv")

# Load the FinBERT model from Hugging Face
sentiment_pipeline = pipeline("text-classification", model="yiyanghkust/finbert-tone")

# Function to classify sentiment of the article content
def get_sentiment(text):
    result = sentiment_pipeline(text[:512])  # FinBERT token limit is 512 tokens
    label = result[0]['label']
    # Classify as 1 (bullish) or 0 (bearish)
    return 1 if label == "positive" else 0

# Apply sentiment classification to each article
df["sentiment"] = df["content"].apply(get_sentiment)

# Save the data with sentiment labels
df.to_csv("finance_news_with_sentiment.csv", index=False)

# Display a preview of the first few rows
df.head()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/533 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/226k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/439M [00:00<?, ?B/s]

Device set to use cpu


,title,url,content,sentiment
0,News,https://www.yahoo.com/,Manage your account 5 min ago Subscribe to...,0
1,Life,https://www.yahoo.com/lifestyle/,"Making friends as an adult can be challenging,...",0
2,Entertainment,https://www.yahoo.com/entertainment/,Daniel Craig isn’t enlisting in the DC Univers...,0
3,Finance,https://finance.yahoo.com/,President Donald Trump suggested that savings ...,0
4,Sports,https://sports.yahoo.com/,On this episode of The Big Number Tom Haberstr...,0


In [17]:
!pip install transformers
!pip install langchain
!pip install torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 89.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 71.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 77.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [18]:
from transformers import pipeline

# Load the pre-trained FinBERT model (sentiment classification)
sentiment_pipeline = pipeline("text-classification", model="yiyanghkust/finbert-tone")

# Function to classify sentiment of the article content
def get_sentiment(text):
    result = sentiment_pipeline(text[:512])  # FinBERT token limit is 512 tokens
    label = result[0]['label']
    # Classify as 1 (bullish) or 0 (bearish)
    return 1 if label == "positive" else 0


Device set to use cpu


In [19]:
# Example list of crawled articles
articles = [
    "The stock market is showing promising growth today.",
    "The market is crashing with huge losses due to inflation concerns.",
    "Tech stocks are doing better despite the economic downturn."
]

# Classify sentiment for each article
sentiments = [get_sentiment(article) for article in articles]

# Print the sentiment results (1 = Bullish, 0 = Bearish)
for article, sentiment in zip(articles, sentiments):
    print(f"Article: {article}\nSentiment: {'Bullish' if sentiment == 1 else 'Bearish'}\n")


Article: The stock market is showing promising growth today.
Sentiment: Bearish

Article: The market is crashing with huge losses due to inflation concerns.
Sentiment: Bearish

Article: Tech stocks are doing better despite the economic downturn.
Sentiment: Bearish



adding experrt opinions:

In [20]:
expert_opinions = {
    "tech stocks": "Analysts believe tech stocks will recover as demand for AI technologies grows.",
    "market downturn": "Experts predict the market will stabilize in the coming months despite the current downturn.",
    "inflation": "Financial analysts are concerned about inflation impacting long-term market growth."
}


In [21]:
def get_expert_opinion(keyword):
    return expert_opinions.get(keyword.lower(), "No expert opinion available.")

# Example article topic
article_topic = "tech stocks"

# Retrieve expert opinion
expert_opinion = get_expert_opinion(article_topic)
print(f"Expert Opinion on {article_topic}: {expert_opinion}")


Expert Opinion on tech stocks: Analysts believe tech stocks will recover as demand for AI technologies grows.


In [22]:
def combine_sentiment_and_expert(sentiment, expert_opinion):
    sentiment_text = "Bullish" if sentiment == 1 else "Bearish"
    return f"Sentiment: {sentiment_text}\nExpert Opinion: {expert_opinion}"

# Example sentiment (from Step 3)
sentiment_result = 1  # Bullish sentiment

# Combine with expert opinion
final_result = combine_sentiment_and_expert(sentiment_result, expert_opinion)
print(final_result)


Sentiment: Bullish
Expert Opinion: Analysts believe tech stocks will recover as demand for AI technologies grows.


adding automation of daily analysis

In [28]:
import yfinance as yf
import time

def get_spy_data():
    spy = yf.Ticker("SPY")
    data = spy.history(period="1d")
    spy_change = data['Close'].pct_change().iloc[-1]
    return spy_change

def get_market_sentiment():
    sentiment = 1  # Mock example (you can replace this with your actual sentiment model)
    return sentiment

def analyze_sentiment_vs_spy(sentiment, spy_change):
    if sentiment == 1:  # Bullish sentiment
        if spy_change > 0:
            return "Positive correlation: Sentiment is Bullish, SPY price increased"
        else:
            return "Negative correlation: Sentiment is Bullish, but SPY price decreased"
    else:  # Bearish sentiment
        if spy_change < 0:
            return "Positive correlation: Sentiment is Bearish, SPY price decreased"
        else:
            return "Negative correlation: Sentiment is Bearish, but SPY price increased"

while True:
    spy_change = get_spy_data()
    market_sentiment = get_market_sentiment()
    correlation_result = analyze_sentiment_vs_spy(market_sentiment, spy_change)

    print(f"SPY Daily Change: {spy_change * 100:.2f}%")
    print(f"Market Sentiment for the Day: {'Bullish' if market_sentiment == 1 else 'Bearish'}")
    print(correlation_result)

    # Wait for the next day
    time.sleep(86400)  # Sleep for 24 hours


YFRateLimitError: Too Many Requests. Rate limited. Try after a while.